## Reading in packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
import pathlib
import shutil
import tempfile
import vaex
from IPython import display
from IPython.display import clear_output
import pyodbc

# Tensorflow related
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow.compat.v2.feature_column as fc

#!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print(tf.__version__)
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)
print(logdir)

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#sys.path.append('...../')

from utils.data_extraction import load_data_final,load_data_and_save
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final,\
                                    generateFeatures_multi_final

from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing,\
                                                    extract_labels_multi_final,\
                                                    align_features_and_labels_multi_final,\
                                                    align_features_and_labels_multi_v5

from utils.models import make_input_fn
from utils.models import performanceTesting,scoreFunction
from utils.plotting import plot_confusion_matrix

2.2.0
C:\Users\PC\AppData\Local\Temp\tmp_cmrr_vf\tensorboard_logs


## Extracting data

In [2]:
# Do we extract new data or read in?
readIn = True
# run load_data()
if readIn:
    
    # Listing the data files 
    path = '../../../Google Drev/Thesis/Data/TAQ/AggregatedTAQ'
#     path = 'F:/AggregatedTAQ/round3'
    datafiles = os.listdir(path)
    content = np.concatenate([['\n\n'],[str(j)+': '+i+'\n' for j,i in enumerate(datafiles) if 'csv' in i],['\n\n']])
    
    # Asking for user input
    file = input('Which one do you want to load? %s'%''.join(content))
    if int(file) <= 2:
        data = pd.read_csv(path + '/' + datafiles[int(file)],
                           header = None,
                           names=['open','high','low','close',
                                  'spread_open','spread_high','spread_low','spread_close',
                                  'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                                  'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                                  'Ticker'])
        # Using the choice of the user to determine the correct market file
        key = re.split('[_.]',datafiles[int(file)])[-2]
        marketDataFile = [file for file in os.listdir(path+'/round5_market_tickers') if key in file]

        # Reading in the market data
        tempData = pd.read_csv(path+'/round5_market_tickers/'+marketDataFile[0]
                               ,header = None
                               ,names=['open','high','low','close',
                                      'spread_open','spread_high','spread_low','spread_close',
                                      'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                                      'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                                      'Ticker'])
        # Adding the market data to the ticker data
        data = pd.concat([data,tempData],axis=0)
    else:
        data = pd.read_csv(path + '/' + datafiles[int(file)],
                           header = 0,
                           index_col=[0,1]
#                            names=['open','high','low','close',
#                                   'spread_open','spread_high','spread_low','spread_close',
#                                   'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
#                                   'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
#                                   'Ticker']
                          )
    
    # Lower casing all column names
#     data.columns = data.columns.str.lower()
else:
    
    # print(os.listdir())
    try:
        path = 'a:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
    except:
        path = 't:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
        
    # Sample type
    data_sample = 'full' # or 'stable'
    # allFiles = os.listdir(path)
    # print(len(allFiles), allFiles[:5], allFiles[-5:])
    # print(allFiles[-10:])

    #dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
    dates = np.array(['20200501']).astype(int)#,'20200402','20200403','20200406','20200407'

    # Provide a list of tickers of interest
    
    tickers = sorted(['TSLA','FB'])#'MSFT'
    
    # Do we need data on trades, quotes or both?
    dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'
    
    if dataNeeded == 'trades':
        tradeData = load_data_final(dates, tickers, dataNeeded, path, verbose)
    elif dataNeeded == 'quotes':
        quoteData = load_data_final(dates,
                                    tickers,
                                    dataNeeded,
                                    path,
                                    verbose,
                                    extract_candles = False,
                                    aggHorizon = 1,
                                    extra_features_from_quotes = None,
                                    data_sample = data_sample)
    elif dataNeeded == 'both':
        tradeData, quoteData = load_data_final(dates, tickers, dataNeeded, path, verbose)

# Reading in sector information
stockInfo = pd.read_csv('../utils/stockInfo_v1.csv',header=[0,1])
stockInfo.columns = ['ticker','sector','exchange','marketCap']

# Creating a table with stock information based on the tickers available in the data.
uniqueTickers = data.Ticker.unique()
stockTable = stockInfo[stockInfo.ticker.isin(uniqueTickers)]
stockTable.head(10)

Which one do you want to load? 

0: aggregateTAQ_May2020_10sec (1).csv
1: aggregateTAQ_May2020_30sec (1).csv
2: aggregateTAQ_May2020_60sec.csv
8: trueAggregateTAQ_60sec.csv


8


C:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticker,sector,exchange,marketCap
12,AAPL,Technology,NMS,1.578173e+12
20,ABBV,Healthcare,NYQ,1.742612e+11
34,ABT,Healthcare,NYQ,1.631410e+11
126,AEP,Utilities,NYQ,4.089551e+10
379,AMT,Real Estate,NYQ,1.171259e+11
428,APD,Basic Materials,NYQ,5.464395e+10
697,BA,Industrials,NYQ,1.020356e+11
699,BABA,Consumer Cyclical,NYQ,5.936536e+11
700,BAC,Financial Services,NYQ,2.020550e+11
870,BHP,Basic Materials,NYQ,1.258194e+11


# Testing the pipeline from Azure

In [3]:
# %%writefile $script_folder/train_nn_keras_v2.py
########## KERAS APPROACH #############

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import numpy as np
import pandas as pd
import argparse
import os
import re
import time
import glob

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout, Activation,LeakyReLU,Flatten
from keras.optimizers import RMSprop, Adam
from keras.callbacks import Callback
from keras.layers.normalization import BatchNormalization


import tensorflow as tf

# from azureml.core import Run

from utils.generate_features import generateFeatures_multi_final
from utils.preprocessing_features_and_labels import align_features_and_labels_multi_final, pre_processing


print("Keras version:", keras.__version__)
print("Tensorflow version:", tf.__version__)

# parser = argparse.ArgumentParser()
# parser.add_argument('--data-source', type=str, dest='data_source', default='data', help='data folder mounting point')
# parser.add_argument('--nn-type', type=str, dest='nn_type', default='ffnn', help='Choose the neural network type')
# parser.add_argument('--batch-size', type=int, dest='batch_size', default=128, help='mini batch size for training')
# parser.add_argument('--first-layer-neurons', type=int, dest='n_hidden_1', default=128,
#                     help='# of neurons in the first layer')
# parser.add_argument('--second-layer-neurons', type=int, dest='n_hidden_2', default=128,
#                     help='# of neurons in the second layer')
# parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.01, help='learning rate')
# parser.add_argument('--dropout-ratio', type=float, dest='dropout_ratio', default=0.0, help='X_1')
# parser.add_argument('--n-layers', type=int, dest='n_layers', default=1, help='X_2')
# parser.add_argument('--l2-penalty', type=float, dest='l2_penalty', default=0.0, help='X_3')
# parser.add_argument('--batch-norm', type=int, dest='batch_norm', default=0, help='X_4')
# parser.add_argument('--batch-shuffle', type=int, dest='batch_shuffle', default=0, help='X_5')
# parser.add_argument('--activation-inner', type=str, dest='activation_inner', default='relu', help='X_6')
# parser.add_argument('--activation-output', type=str, dest='activation_output', default='softmax', help='X_7')

# ## Newly added
# parser.add_argument('--featureset', type=int, dest='feature_set',default=1, help='specifying the feature set')
# parser.add_argument('--pre-processing', type=str, dest = 'pre_processing',default=None, help='pre-processing of training and validation data')
# parser.add_argument('--feature-lags', type=int, dest='feature_lags', default=0, help='number of lagged features')
# parser.add_argument('--n-epochs', type=int, dest = 'n_epochs', default=20, help='number of epochs to run')
# parser.add_argument('--label-type', type=int, dest = 'label_type', default=1, help='label type')
# parser.add_argument('--pastobs-in-percentage', type=int, dest = 'pastobs_in_percentage', default=0, help='Option to specify if the pastobs feature should be in percentage')
# parser.add_argument('--resume-from', type=str, default=None,
#                     help='location of the model or checkpoint files from where to resume the training')
# args = parser.parse_args()

# previous_model_location = args.resume_from
n_feature_lags = 1
features_to_use = 1
pre_procesing_applied = 'quantgau'
n_epochs = 2
label_type = 2
pastobs_in_percentage = 1
nn_type = 'lstm'
dropout_ratio = 0.5
n_layers = 3
# l2_penalty = 
batch_norm = 1
batch_shuffle = 0
activation_inner = 'leakyrelu'
activation_output = 'softmax'
n_h1 = 128
n_h2 = 128
batch_size = 21450
learning_rate = 0.001

# start an Azure ML run
# run = Run.get_context()

# datasetName = 'aggregatedTAQ_60sec_wsectors'

# get the input dataset by name
# dataset = run.input_datasets[datasetName] # [data_source]

### load train and test set into numpy arrays
# load the TabularDataset to pandas DataFrame
# data = dataset.to_pandas_dataframe().set_index(['Column1','Column2'])
# data.columns = ['open','high','low','close','spread_open',
#                               'spread_high','spread_low','spread_close',
#                               'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
#                               'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
#                               'Ticker','sector']

# Removing the XNTK ticker
data = data[~data.Ticker.isin(['XNTK'])]

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF','SPY','DIA','QQQ']

# Extracting the sector ETFs to a separate variable
sectorETFS = data[data.Ticker.isin(etfs)]

# Removing the ETFs
data = data[~data.Ticker.isin(etfs)]



########### Generate Features ################

# feature_sets = {0:'base',
#                 1:'base with sectors',
#                 # 2:'technical',
#                 # 3:'technical with sectors'
#                 2:'base with technical',
#                 # 3:'base with techinical and sectors',
#                 3:'full'}

if features_to_use == 0:
    listOfFeatures = [
                        'pastobs',
                        'spread',
                        'bidsize',
                        'ofrsize',
                        'pastreturns',
                        'intradaytime'
                    ]

elif features_to_use == 1:

    listOfFeatures = [
                    'pastobs',
                    'spread',
                    'bidsize',
                    'ofrsize',
                    'pastreturns',
                    'intradaytime',
                    'sector'
                    ]

elif features_to_use == 2:
    listOfFeatures = [
                        'pastobs',
                        'spread',
                        'bidsize',
                        'ofrsize',
                        'pastreturns',
                        'intradaytime',
                        'stok',
                        'stod',
                        'sstod',
                        # 'wilr',
                        'roc',
                        'rsi',
                        'atr',
                        'cci',
                        'dpo',
                        'sma',
                        'ema',
                        'macd',
                        'macd_diff',
                        'macd_signal',
                        'dis5',
                        'dis10'
                        ]

elif features_to_use == 3:
 
    listOfFeatures = [
                    'pastobs',
                    'spread',
                    'bidsize',
                    'ofrsize',
                    'pastreturns',
                    'intradaytime',
                    'stok',
                    'stod',
                    'sstod',
                    # 'wilr',
                    'roc',
                    'rsi',
                    'atr',
                    'cci',
                    'dpo',
                    'sma',
                    'ema',
                    'macd',
                    'macd_diff',
                    'macd_signal',
                    'dis5',
                    'dis10',
                    'sector'
                    ]

elif features_to_use == 4:
    raise NotImplementedError
    ##

elif features_to_use == 5:
    raise NotImplementedError
    ##

elif features_to_use == 6:
    raise NotImplementedError

## Extracting the features

features = generateFeatures_multi_final(data = data, 
                                        listOfFeatures = listOfFeatures, 
                                        feature_lags = n_feature_lags,
                                        sectorETFS=sectorETFS,
                                        pastobs_in_percentage=pastobs_in_percentage)


########### Generate Labels ################

# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# Label options:

'''
Label construction:
- Binary median split
	- positive/negative returns in our case, at 1 minute horizon.

- Multi 33-33-33 chunks

- Multi 20-60-20 chunks

- Multi 20-20-20-20-20 chunks

- Multi 10-20-40-20-10 chunks
'''

if label_type == 0:
    n_classes = 2
    label_split = [] # empty means equal split

elif label_type == 1:
    n_classes = 3
    label_split = [] # empty means equal split    

elif label_type == 2:
    n_classes = 3
    label_split = [0, 0.2, 0.8, 1] # for non-equal, include 0 and 1     

elif label_type == 3:
    n_classes = 5
    label_split = [] # empty means equal split        

elif label_type == 4:
    n_classes = 5
    label_split = [0, 0.1, 0.3, 0.7, 0.9, 1] # for non-equal, include 0 and 1

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y, indices = align_features_and_labels_multi_final(price_candles = price_candles, 
                                                        all_features = features,
                                                        prediction_horizon = 1, 
                                                        n_feature_lags = n_feature_lags, 
                                                        n_classes = n_classes, # 5,
                                                        label_split = label_split,
                                                        safe_burn_in = False, 
                                                        data_sample = 'full',
                                                        splitType='global',
                                                        noise=False,
                                                        ticker_dummies=False)


# Adding ticker dummies
tickers = X.pop('ticker')
X = pd.concat([X, pd.get_dummies(tickers, prefix='d_ticker', drop_first=False)], axis=1)

########### Splitting data ################

# Let's have a proper split (along tickers & dates)
train_size = 0.8

# Sort the indices
tempIndices = indices.sort_values(['days','timestamps','ticker'])

# Sorting the data
X = X.loc[tempIndices.index,:]#.head(66)
y = y.loc[tempIndices.index,:]

# extracting the first date for the validation data.
first_val_day = int(np.floor(indices.days.unique().shape[0]*0.8))

# Splitting the data
X_train = X[tempIndices.days<tempIndices.days.unique()[first_val_day]]
y_train = y[tempIndices.days<tempIndices.days.unique()[first_val_day]]

X_test = X[tempIndices.days>=tempIndices.days.unique()[first_val_day]]
y_test = y[tempIndices.days>=tempIndices.days.unique()[first_val_day]]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

training_set_size = X_train.shape[0]


########### Pre-processing: none right now ################

if  pre_procesing_applied == 'None':
    # do nothing here
    pass

elif  pre_procesing_applied == 'std':

    # Standardize some features
    ppdict1 = {i:'std' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'minmax':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'minmax' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

elif pre_procesing_applied == 'pow':

    # Standardize some features
    ppdict1 = {i:'pow' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

elif pre_procesing_applied == 'quantgau':

    # Standardize some features
    ppdict1 = {i:'quantgau' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

elif pre_procesing_applied == 'individual':

    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

elif pre_procesing_applied == 'stacked':
    
    for j in ['pow','std','minmax']:

        ppdict1 = {i:j for i in X_train.columns if 'd_' != i[0:2]}

        # Keep some in actual levels (Dummies in this case).
        ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

        # Merging the two
        ppdict = {**ppdict1,**ppdict2}

        X_train,X_test = pre_processing(X_train,X_test,ppdict)

if pre_procesing_applied not in ['None','stacked']:
    X_train,X_test = pre_processing(X_train,X_test,ppdict)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

training_set_size = X_train.shape[0]

n_inputs = X_train.shape[1] #28 * 28
n_outputs = n_classes # 2 
n_epochs = n_epochs# 20

# one-hot encode a 1-D array
def one_hot_encode(array, num_of_classes):
    return np.eye(num_of_classes)[array.reshape(-1)]

y_train = one_hot_encode(y_train.values.astype(np.int), n_outputs)
y_test = one_hot_encode(y_test.values.astype(np.int), n_outputs)

print('Shapes after one-hot-encoding labels')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

if nn_type =='ffnn':
    # Build a simple MLP model
    model = Sequential()

    for layer in np.arange(n_layers):
        if layer == 0:
            # first hidden layer
            model.add(Dense(n_h1, input_shape=(n_inputs,)))#, activation=activation_inner
            
            if batch_norm:
                model.add(BatchNormalization())
            
            if activation_inner == 'leakyrelu':
                model.add(LeakyReLU())
            else:
                model.add(Activation(activation_inner))
            # dropout
            if dropout_ratio > 0:
                model.add(Dropout(dropout_ratio))
        else:

            # second hidden layer
            model.add(Dense(n_h2))
            if batch_norm:
                model.add(BatchNormalization())
            
            if activation_inner == 'leakyrelu':
                model.add(LeakyReLU())
            else:
                model.add(Activation(activation_inner))

            # dropout
            if dropout_ratio > 0:
                model.add(Dropout(dropout_ratio))

    
    # output layer
    model.add(Dense(n_outputs))
    
    if batch_norm:
        model.add(BatchNormalization())
            
    model.add(Activation(activation_output))

elif nn_type == 'lstm':
    
    # LSTM Data Prep
    # reshape input to be 3D [samples, timesteps, features]
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # LSTM
    model = Sequential()

    for layer in np.arange(n_layers):

        if layer==0:

            #Adding the first LSTM layer and some Dropout regularisation
            # model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
            model.add(LSTM(units = n_h1//2, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2]))) # KRISNI: changed X_train.shape[1] to X_train.shape[0] - Update: Changed it back
            
            if batch_norm:
                model.add(BatchNormalization())
            
            # model.add(Activation(activation_inner))

            # dropout
            if dropout_ratio > 0:
                model.add(Dropout(dropout_ratio))

        elif layer < (n_layers-1):
            # Adding a second LSTM layer and some Dropout regularisation
            model.add(LSTM(units = n_h2//2, return_sequences = True))
            if batch_norm:
                model.add(BatchNormalization())
            
            # model.add(Activation(activation_inner))

            # dropout
            if dropout_ratio > 0:
                model.add(Dropout(dropout_ratio))

        else:
            # Adding a second LSTM layer and some Dropout regularisation
            model.add(LSTM(units = n_h2//2, return_sequences = False))
            if batch_norm:
                model.add(BatchNormalization())
            
            # model.add(Activation(activation_inner))

            # dropout
            if dropout_ratio > 0:
                model.add(Dropout(dropout_ratio))

    # Adding the output layer
#     model.add(Flatten()) # New - to combat the latest error, logged in azure-to-do file
    model.add(Dense(units = n_outputs))

    if batch_norm:
        model.add(BatchNormalization())
            
    model.add(Activation(activation_output))


model.summary()

model.compile(loss='categorical_crossentropy', #'sparse_categorical_crossentropy',
              optimizer=Adam(lr=learning_rate),
              metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']) #metrics = ['accuracy']) #) # #

# start an Azure ML run
# run = Run.get_context()


class LogRunMetrics(Callback):
    # callback at the end of every epoch
    def on_epoch_end(self, epoch, log):
        # log a value repeated which creates a list
        run.log('Loss', log['val_loss'])
        run.log('Accuracy', log['val_accuracy'])
        run.log('AUC', log['val_auc'])

        run.log('Train Loss', log['loss'])
        run.log('Train Accuracy', log['accuracy'])
        run.log('Train AUC', log['auc'])        


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=n_epochs,
                    verbose=2,
                    validation_data=(X_test, y_test),
#                     callbacks=[LogRunMetrics()],
#                     shuffle=batch_shuffle
                    )

score = model.evaluate(X_test, y_test, verbose=0)

print("Scores", score)

# # log a single value
# run.log("Final test loss", score[0])
print('Test loss:', score[0])

# run.log('Final test AUC', score[1])
print('Test AUC:', score[1])

# run.log('Final test accuracy', score[2])
print('Test accuracy:', score[2])

# # # log an image
# # run.log_image('Accuracy vs Loss', plot=plt)

# # create a ./outputs/model folder in the compute target
# # files saved in the "./outputs" folder are automatically uploaded into run history
# os.makedirs('./outputs/model', exist_ok=True)

# # serialize NN architecture to JSON
# model_json = model.to_json()
# # save model JSON
# with open('./outputs/model/model.json', 'w') as f:
#     f.write(model_json)
# # save model weights
# model.save_weights('./outputs/model/model.h5')
# print("model saved in ./outputs/model folder")



Keras version: 2.4.3
Tensorflow version: 2.2.0


INFO:MainThread:numexpr.utils:NumExpr defaulting to 4 threads.
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


AAPL done
ABBV done
ABT done
AEP done
AMT done
APD done
BA done
BABA done
BAC done
BHP done
BP done
CCI done
CHL done
COST done
CSGP done
D done
DIS done
ECL done
ENB done
EXC done
FB done
FMX done
GOOG done
INTC done
JNJ done
KO done
LFC done
LIN done
LMT done
MA done
MCD done
MSFT done
NKE done
NVDA done
NVS done
Number of NaNs in label: 1. 1 is expected
Returns that lead to NaNs in label: [0.0907158]
PBR done
PEP done
PFE done
PLD done
PSA done
PTR done
PYPL done
RTX done
SHW done
SNP done
SO done
SRE done
T done
TM done
TSLA done
TSM done
UNP done
UPS done
V done
WMT done
(343200, 130)
(343200, 1)
(85690, 130)
(85690, 1)
(343200, 130)
(343200, 1)
(85690, 130)
(85690, 1)
Shapes after one-hot-encoding labels
(343200, 130)
(343200, 3)
(85690, 130)
(85690, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 64)             49920     
____________

InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential/activation/Softmax:0) = ] [[nan nan nan]...] [y (Cast_2/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/Assert}}]] [Op:__inference_train_function_8582]

Function call stack:
train_function


In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
y_train.shape

In [ ]:
stop


# reading in the market data (done automatically atm)

In [ ]:
file

In [ ]:
data

In [ ]:
data

In [ ]:
data.columns

### Dropping ETFS and market indices

In [ ]:
data.Ticker.unique()

In [ ]:
# Removing the XNTK ticker
data = data[~data.Ticker.isin(['XNTK'])]

In [ ]:
data.Ticker.unique()

In [ ]:
# Removing the XNTK ticker
data = data[~data.Ticker.isin(['XNTK'])]

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF','SPY','DIA','QQQ']

# Extracting the sector ETFs to a separate variable
sectorETFS = data[data.Ticker.isin(etfs)]

# Removing the ETFs
data = data[~data.Ticker.isin(etfs)]

In [ ]:
data.columns

In [ ]:
data

In [ ]:
########### Generate Features ################

n_feature_lags = 1

# features = generateFeatures_multi_final(data = data, 
#                                   listOfFeatures = [
#                                                     'pastobs',
#                                                     'spread',
#                                                     'bidsize',
#                                                     'ofrsize',
# #                                                     'stok',
# #                                                     'stod',
# #                                                     'sstod',
# #                                                     'wilr',
# #                                                     'roc',
# #                                                     'rsi',
# #                                                     'atr',
# #                                                     'cci',
# #                                                     'dpo',
# #                                                     'sma',
# #                                                     'ema',
# #                                                     'macd',
# #                                                       'macd_diff',
# #                                                       'macd_signal',
# #                                                     'dis5',
# #                                                     'dis10',
#                                                       'sector'
#                                                    ], 
#                                    feature_lags = n_feature_lags
#                                      ,stockTable=stockTable)
features = generateFeatures_multi_final(data = data, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'spread',
                                                    'bidsize',
                                                    'ofrsize',
                                                    'stok',
                                                    'stod',
                                                    'sstod',
#                                                     'wilr',
                                                    'roc',
                                                    'rsi',
                                                    'atr',
                                                    'cci',
                                                    'dpo',
                                                    'sma',
                                                    'ema',
                                                    'macd',
                                                      'macd_diff',
                                                      'macd_signal',
                                                    'dis5',
                                                    'dis10',
                                                      'sector'
                                                   ], 
                                   feature_lags = n_feature_lags
                                     ,sectorETFS=sectorETFS)

########### Generate Labels ################

n_classes = 2
# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y,indices = align_features_and_labels_multi_final(price_candles = price_candles, 
                                                 all_features = features,
                                                 prediction_horizon = 1, 
                                                 n_feature_lags = n_feature_lags, 
                                                 n_classes = n_classes, # 5,
                                                 safe_burn_in = False, 
                                                 data_sample = 'full',
                                                 splitType='global',
                                                 noise=False,ticker_dummies=False)

## Splitting the data

## Adding ticker dummies

In [ ]:
## Adding ticker dummies
tickers = X.pop('ticker')
X = pd.concat([X, pd.get_dummies(tickers, prefix='ticker', drop_first=False)], axis=1)

In [ ]:
X.columns

## Constructing our final train/validation sets

In [ ]:
# train_ds = pd.concat([X.iloc[start:end, :] for (start, end) in train_ranges]).reset_index(drop=True)
# train_y = pd.concat([y.iloc[start:end] for (start, end) in train_ranges]).reset_index(drop=True)

# validate_ds = pd.concat([X.iloc[start:end, :] for (start, end) in val_ranges]).reset_index(drop=True)
# val_y = pd.concat([y.iloc[start:end] for (start, end) in val_ranges]).reset_index(drop=True)

# train_ds.shape, train_y.shape, validate_ds.shape, val_y.shape, train_y.shape[0] + val_y.shape[0]

# Let's have a proper split (along tickers & dates)
train_size = 0.8

# Sort the indices
tempIndices = indices.sort_values(['days','timestamps','ticker'])

# Sorting the data
X = X.loc[tempIndices.index,:]#.head(66)
y = y.loc[tempIndices.index,:]

# extracting the first date for the validation data.
first_val_day = int(np.floor(indices.days.unique().shape[0]*0.8))

# Splitting the data
X_train = X[tempIndices.days<tempIndices.days.unique()[first_val_day]].reset_index(drop=True)
y_train = y[tempIndices.days<tempIndices.days.unique()[first_val_day]].reset_index(drop=True)

X_test = X[tempIndices.days>=tempIndices.days.unique()[first_val_day]].reset_index(drop=True)
y_test = y[tempIndices.days>=tempIndices.days.unique()[first_val_day]].reset_index(drop=True)


print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

In [ ]:
X_train

In [ ]:
np.where((np.sum(np.isinf(X_train.values), axis=1) == 0) == False),
np.where((np.sum(np.isnan(X_train.values), axis=1) == 0) == False)#X_train


## Pre-processing

In [ ]:
{i:colname for i,colname in enumerate(train_ds.columns)}

In [ ]:
# Creating one ppdict for individual preprocessings
# ppdict1 = {'open':'minmax',
#           'high':'log',
#           'low':'log',
#           'close':'std'}
# splitpoint = 32

# # Standardize some features
# ppdict1 = {i:'std' for i in train_ds.columns[0:splitpoint]} 
# # Keep some in actual levels (Dummies in this case).
# ppdict2 = {i:'act' for i in train_ds.columns[splitpoint:]}

pre_procesing_applied = 'std'

# Merging the two
# ppdict = {**ppdict1,**ppdict2}

if  pre_procesing_applied == 'None':
    # do nothing here
    pass

elif  pre_procesing_applied == 'std':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'std' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'minmax':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'minmax' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(X_train,X_test,pp_dict)

elif pre_procesing_applied == 'pow':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'pow' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'quantgau':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'quantgau' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'individual':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    
    # ppdict1 = {i:'power' for i in X_train.columns if 'd_' != i[0:2]}


    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'stacked':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    
    for j in ['pow','std','minmax']:

        ppdict1 = {i:j for i in X_train.columns if 'd_' != i[0:2]}

        # Keep some in actual levels (Dummies in this case).
        ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

        # Merging the two
        ppdict = {**ppdict1,**ppdict2}

        X_train,X_test = pre_processing(X_train,X_test,ppdict)

if pre_procesing_applied not in ['None','stacked']:
    X_train,X_test = pre_processing(X_train,X_test,ppdict)

In [ ]:
ppX_train.iloc[:,0].mean(),ppX_train.iloc[:,0].std()

## Prepping for models

In [ ]:
N_VALIDATION = val_y.shape[0] #int(1e3)
N_TRAIN = train_y.shape[0] #int(1e4)
# BUFFER_SIZE = int(1e4)
BATCH_SIZE = 256 #512 #32
MAX_EPOCHS = 500

STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_REPEAT = int(N_TRAIN / ((STEPS_PER_EPOCH * MAX_EPOCHS) / BATCH_SIZE))
FEATURES = X.shape[1]

N_TRAIN, N_VALIDATION, N_TRAIN + N_VALIDATION, STEPS_PER_EPOCH, N_REPEAT, STEPS_PER_EPOCH * MAX_EPOCHS

## A Logistic Regression model in TF/Keras

In [ ]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      #keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# def make_model(metrics = METRICS, output_bias=None):
#   if output_bias is not None:
#     output_bias = tf.keras.initializers.Constant(output_bias)
#   model = keras.Sequential([
#       keras.layers.Dense(
#           16, activation='relu',
#           input_shape=(train_features.shape[-1],)),
#       keras.layers.Dropout(0.5),
#       keras.layers.Dense(1, activation='sigmoid',
#                          bias_initializer=output_bias),
#   ])

#   model.compile(
#       optimizer=keras.optimizers.Adam(lr=1e-3),
#       loss=keras.losses.BinaryCrossentropy(),
#       metrics=metrics)

#   return model

# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,))
# })

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10)
    keras.layers.Dense(1,
                       input_shape=(FEATURES,),
                       activation='sigmoid',
                       kernel_regularizer=regularizers.l2(1))
])

model.summary()

# with final activation (Keras/TF tutorial advises against this practice, but they also use it later in the tutorial)
# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,), activation='sigmoid')
# })

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', ])
model.compile(
              optimizer=keras.optimizers.Adam(), #lr=1e-3
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)

early_stopping = tf.keras.callbacks.EarlyStopping(
                                                monitor='val_auc', 
                                                verbose=1,
                                                patience=100,
                                                mode='max',
                                                restore_best_weights=True)

def get_callbacks(run_id):
      return [
             tfdocs.modeling.EpochDots(),
             early_stopping,
             tf.keras.callbacks.TensorBoard(logdir), #/run_id),
      ]

baseline_history = model.fit(
                            train_ds, #train_features,
                            train_y, #train_labels,
                            batch_size=512, #BATCH_SIZE,
                            epochs=1000, #EPOCHS,
                            callbacks = get_callbacks(run_id = 'first'), #[early_stopping],
                            validation_data=(validate_ds, val_y),
                            verbose=0) #(val_features, val_labels))

In [ ]:
model.evaluate(validate_ds,  val_y, verbose=2)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
%tensorboard --logdir logs